In [1]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from algo_pre_processing import ProjectsData
import pandas as pd

from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
from sklearn.model_selection import KFold
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


warnings.filterwarnings("ignore")

In the first section, you will load the projects' data. After that, you can proceed to run the statistical feature experiment and the embedding-based features experiment.

Since we used 2 batches of data, you will find `embedding_features1` and `embedding_features2` (or `statistical_features1` and `statistical_features2`) representing the two batches we used. However, if you are running an experiment with only one batch of data, you can choose to use only one of them. Simply assign the final dataset (`embedding_features` or `statistical_features`) as the dataset you created.

## Load projects' data

Change the following paths according to your implementation:
- ProjectsData Object creation:
    - `MCW_path`: the results of the label extraction part - the MCW run.
    - `projects_path`: the path of the arff files as defined in section 2 of label extraction, you can use either `arff_source` folder or `arff_dest` folder.
    - `features_path`: the path of the features you would like to use, for the statistical meta-features we used the output of `statistical meta features.py` and for the embedding based meta-features we used the output of the `embedding java files` folder instructions. 
    - `algo_matlab_results`: the results of the label extraction part - the MATLAB algorithms output.
    - `to_use`: select the folders numbers that you want to use - according to the number of groups defined in `random_groups.py`.
- Get data function: use mode for majority voting labeling.

In [2]:
# get project's data
projects_data = ProjectsData(MCW_path= 'data/MCW/batch1',
                 projects_path= 'data/Projects_Arff_batch1',
                 features_path= 'meta_model_features/embeddings/first_batch/base_buggy_features.csv',
                 algo_matlab_results= 'data/Results_Rest_batch1',
                 to_use=[1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35,
 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50])
embedding_features1 = projects_data.get_data(label_method='mode')
algo_list = projects_data.get_algo_list()
algo_scores = projects_data.get_algo_scores()
projects_data2 = ProjectsData(MCW_path='data/MCW/batch2',
                 projects_path='data/Projects_Arff_batch2',
                 features_path= 'meta_model_features/embeddings/second_batch/base_buggy_features.csv',
                 algo_matlab_results='data/Results_Rest_batch2',
                 to_use=list(range(1, 100)))
embedding_features2 = projects_data2.get_data(label_method='mode')

embedding_features = pd.concat([embedding_features1,embedding_features2]) # embedding

Loading projects is finished...
Loading projects is finished...


In [3]:
# get project's data
projects_data = ProjectsData(MCW_path= 'data/MCW/batch1',
                 projects_path= 'data/Projects_Arff_batch1',
                 features_path= 'meta_model_features/statistical_features/features_first_batch.csv',
                 algo_matlab_results= 'data/Results_Rest_batch1',
                            to_use=[1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35,
 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50])
statistical_features1 = projects_data.get_data(label_method='mode')
algo_list = projects_data.get_algo_list()
algo_scores = projects_data.get_algo_scores()
projects_data2 = ProjectsData(MCW_path='data/MCW/batch2',
                 projects_path='data/Projects_Arff_batch2',
                 features_path= 'meta_model_features/statistical_features/features_second_batch.csv',
                 algo_matlab_results='data/Results_Rest_batch2',
                 to_use=list(range(1, 100)))
statistical_features2 = projects_data2.get_data(label_method='mode')

statistical_features = pd.concat([statistical_features1,statistical_features2]) # statistical features

Loading projects is finished...
Loading projects is finished...


In [4]:
# make sure we have the same projects for embedding experiment and statistical experiment
pv_embedding = embedding_features[['pv']]
pv_embedding.columns = ['pv2']
statistical_features = statistical_features.merge(pv_embedding, left_on='pv', right_on='pv2')
statistical_features = statistical_features.drop(columns=['mode_algo'])
statistical_features = projects_data.get_mode_value(statistical_features)

In [5]:
def results_processing(algo_list, joined, y_test, preds):
    # get the predictions of the algo + the real label
    new_df = pd.DataFrame(y_test)
    new_df['prediction'] = preds
    with_res = new_df.join(joined, lsuffix="l")
    with_res = with_res[['prediction', 'best_algo'] + algo_scores]

    for metric in ["", "_precision", "_recall", "_accuracy"]:
        # get comb results
        with_res['comb' + metric] = None
        for algo in algo_list:
            with_res['comb' + metric].loc[with_res['prediction'] == algo] = \
                with_res[with_res['prediction'] == algo][
                    algo + metric]
        with_res['comb' + metric] = with_res['comb' + metric].astype(float)

    # get best algorithm results
    with_res['best'] = None
    for algo in algo_list:
        with_res['best'].loc[with_res['best_algo'] == algo] = with_res[with_res['best_algo'] == algo][algo]
    with_res['best'] = with_res['best'].astype(float)

    # get the error for each algo
    for algo in algo_list + ['comb']:
        with_res[algo + "_error"] = (with_res['best'] - with_res[algo]).astype(float)
    return with_res
random_st = 14

grid_dictionary = {'RF': (RandomForestClassifier(random_state=random_st), 
                          {'n_estimators': [50,100,150],
                            'max_features': ['sqrt', 'log2'],
                            'max_depth': [2,3],
                            'criterion': ['gini', 'entropy', "log_loss"]}),
                   'XGboost': (GradientBoostingClassifier(random_state=random_st),
                               {"loss": ["deviance"],
                                "learning_rate": [0.05,0.1],
                                "min_samples_split": [0.28,0.4],
                                "min_samples_leaf": [0.1,0.2],
                                "max_depth": [2,3],
                                "max_features": ["sqrt"],
                                "criterion": ["friedman_mse","mse"],
                                "subsample": [0.5,0.8],
                                "n_estimators": [50,100],
                                "warm_start": [True, False]}),
                  'LR': (LogisticRegression(random_state=random_st),
                        {
                                'penalty' : ['l1','l2'], 
                                'C'       : np.logspace(-3,3,7),
                                'solver'  : ['newton-cg', 'lbfgs', 'liblinear'],}),
                  'SVM': (SVC(),
                                {'C': [0.1, 1, 10, 100, 1000], 
                                  'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                                  'kernel': ['rbf']} )
}

## Statistical features experiment

1. Select the meta classifier that will be used: RF-Random Forest, XGboost, LR-Logistic Regression or SVM-Support Vector Machine.
2. Select the number of folds for the cross validation.

In [6]:
META_CLASSIFIER = 'XGboost'
NUM_OF_FOLDS = 5

In [7]:
statistical_features = statistical_features[statistical_features['best_algo'] != 'Dycom']
statistical_features = statistical_features[statistical_features['best_algo'] != 'LT']
X = statistical_features.drop(columns=['pv'] + algo_scores + ['best_algo','mode_algo','pv2','Unnamed: 0_std','Unnamed: 0_avg', 'Unnamed: 0_min','Unnamed: 0_skew'])
X = X.fillna(0)
y = statistical_features['best_algo']

In [8]:
statistical_features.drop_duplicates()['best_algo'].value_counts()

MCW        548
TPTL        93
TDS         43
TCA_rnd     19
Name: best_algo, dtype: int64

In [9]:
acc_list_stat, f1_list = [], []
# split to train and test + oversampling
mean_results_stat = {'best':[],'MCW': [], 'MCW_precision': [], 'MCW_recall': [], 'MCW_accuracy': [], 'comb': [], 'comb_precision': [], 'comb_recall': [],
 'comb_accuracy': [], 'TPTL': [], 'TPTL_precision': [], 'TPTL_recall': [], 'TPTL_accuracy': [], 'TCA_rnd': [], 'TCA_rnd_precision': [],
 'TCA_rnd_recall': [], 'TCA_rnd_accuracy': [], 'LT': [], 'LT_precision': [], 'LT_recall': [], 'LT_accuracy': [], 'Dycom': [], 'Dycom_precision': [],
 'Dycom_recall': [], 'Dycom_accuracy': [], 'TDS': [], 'TDS_precision': [], 'TDS_recall': [], 'TDS_accuracy': []}

error_results = {'MCW': [], 'comb': [], 'TPTL': [], 'TCA_rnd': [], 'LT': [], 'Dycom': [], 'TDS': []}
ss = KFold(n_splits=NUM_OF_FOLDS,shuffle=True,random_state=14)

fold = 0
for train_index, test_index in ss.split(X,y):
    fold += 1
    scaler = StandardScaler()
    x_train, y_train = X.iloc[train_index], y.iloc[train_index]
    x_test, y_test = X.iloc[test_index], y.iloc[test_index]
    sm = SMOTE(random_state=random_st, k_neighbors=2)
    X_samp, y_samp = sm.fit_resample(x_train, y_train)
    print("Oversampling is finished....")
    # building the model
    rfc = grid_dictionary[META_CLASSIFIER][0]
    param_grid = grid_dictionary[META_CLASSIFIER][1]

    CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
    CV_rfc.fit(X_samp, y_samp)

    preds = CV_rfc.best_estimator_.predict(x_test)
    print(CV_rfc.best_estimator_)
    acc_list_stat.append(accuracy_score(y_test, preds))
    f1_list.append(f1_score(y_test, preds, average='micro'))

    ####### RESULTS ANALYSIS #######
    with_res = results_processing(algo_list, statistical_features, y_test, preds)
    
    for algo in ['MCW', 'comb', 'TPTL', 'TCA_rnd', 'LT', 'Dycom', 'TDS']:
        mean_results_stat[algo].append(np.mean(with_res[algo]))
        mean_results_stat[algo + "_precision"].append(np.mean(with_res[algo + "_precision"]))
        mean_results_stat[algo + "_recall"].append(np.mean(with_res[algo + "_recall"]))
        mean_results_stat[algo + "_accuracy"].append(np.mean(with_res[algo + "_accuracy"]))
        error_results[algo].append(np.mean(with_res[algo + "_error"]))
    mean_results_stat['best'].append(np.mean(with_res['best']))
    with_res.to_csv(f"Results_meta_model/current_results/statistical_result_{fold}.csv")
    
print('random_state:', random_st)
print("MCW f1: ", np.mean(mean_results_stat['MCW']), ", MCW acc: ", np.mean(mean_results_stat['MCW_accuracy']),
      ", MCW precision: ", np.mean(mean_results_stat['MCW_precision']), ", MCW recall: ",
      np.mean(mean_results_stat['MCW_recall']))
print("comb f1: ", np.mean(mean_results_stat['comb']), ", comb acc: ", np.mean(mean_results_stat['comb_accuracy']),
      ", comb precision: ", np.mean(mean_results_stat['comb_precision']), ", comb recall: ",
      np.mean(mean_results_stat['comb_recall']))
print("best: ", np.mean(mean_results_stat['best']))
print("accuracy meta model list: " , acc_list_stat)

Oversampling is finished....
GradientBoostingClassifier(loss='deviance', max_features='sqrt',
                           min_samples_leaf=0.1, min_samples_split=0.28,
                           random_state=14, subsample=0.8, warm_start=True)
Oversampling is finished....
GradientBoostingClassifier(loss='deviance', max_features='sqrt',
                           min_samples_leaf=0.1, min_samples_split=0.28,
                           random_state=14, subsample=0.8, warm_start=True)
Oversampling is finished....
GradientBoostingClassifier(loss='deviance', max_features='sqrt',
                           min_samples_leaf=0.1, min_samples_split=0.28,
                           random_state=14, subsample=0.8, warm_start=True)
Oversampling is finished....
GradientBoostingClassifier(loss='deviance', max_features='sqrt',
                           min_samples_leaf=0.1, min_samples_split=0.28,
                           random_state=14, subsample=0.8, warm_start=True)
Oversampling is finished....

In [11]:
# pd.set_option('max_columns',30)
pd.DataFrame(mean_results_stat)

,best,MCW,MCW_precision,MCW_recall,MCW_accuracy,comb,comb_precision,comb_recall,comb_accuracy,TPTL,...,LT_recall,LT_accuracy,Dycom,Dycom_precision,Dycom_recall,Dycom_accuracy,TDS,TDS_precision,TDS_recall,TDS_accuracy
0,0.330294,0.307057,0.308100,0.384835,0.892901,0.300967,0.381728,0.360887,0.845211,0.170108,...,0.162853,0.904410,0.108718,0.281733,0.123286,0.738009,0.138784,0.548405,0.091758,0.599550
1,0.346266,0.309555,0.326753,0.353262,0.883733,0.312707,0.419149,0.332698,0.839235,0.186643,...,0.145573,0.891910,0.119757,0.282359,0.124523,0.730365,0.165816,0.545690,0.117522,0.623025
2,0.323066,0.295366,0.285046,0.361477,0.876504,0.308079,0.387107,0.359182,0.820011,0.181867,...,0.158895,0.894400,0.115055,0.262673,0.133048,0.745068,0.153868,0.518993,0.113717,0.619589
3,0.295705,0.260186,0.269828,0.320819,0.885216,0.270314,0.364903,0.302682,0.844962,0.169433,...,0.165044,0.906480,0.106914,0.259459,0.115239,0.762344,0.154110,0.555324,0.103495,0.650073
4,0.325609,0.297082,0.291640,0.357392,0.882825,0.302692,0.375003,0.346093,0.827697,0.166095,...,0.164099,0.899776,0.098842,0.249429,0.120544,0.738947,0.146722,0.567562,0.099145,0.608196


In [12]:
res_unioned = []
for i in range(1,6):
    temp = pd.read_csv(f'Results_meta_model/current_results/statistical_result_{i}.csv')
    res_unioned.append(temp)
res_pd = pd.concat(res_unioned)
res_pd.to_csv('statistical_results.csv')

## Embedding based features experiment

1. Select the meta classifier that will be used: RF-Random Forest, XGboost, LR-Logistic Regression or SVM-Support Vector Machine.
2. Select the number of folds for the cross validation.

In [13]:
META_CLASSIFIER = 'XGboost'
NUM_OF_FOLDS = 5

In [14]:
embedding_features = embedding_features.drop(columns=['mode_algo'])
embedding_features = projects_data.get_mode_value(embedding_features)
embedding_features = embedding_features[embedding_features['best_algo'] != 'Dycom']
embedding_features = embedding_features[embedding_features['best_algo'] != 'LT']
X = embedding_features.drop(columns=['pv'] + algo_scores + ['best_algo','mode_algo'])
X = X.fillna(0)
y = embedding_features['best_algo']

In [15]:
embedding_features.drop_duplicates()['best_algo'].value_counts()

MCW        548
TPTL        93
TDS         43
TCA_rnd     19
Name: best_algo, dtype: int64

In [17]:
acc_list_embd, f1_list = [], []
# split to train and test + oversampling
mean_results_embd = {'best':[],'MCW': [], 'MCW_precision': [], 'MCW_recall': [], 'MCW_accuracy': [], 'comb': [], 'comb_precision': [], 'comb_recall': [],
 'comb_accuracy': [], 'TPTL': [], 'TPTL_precision': [], 'TPTL_recall': [], 'TPTL_accuracy': [], 'TCA_rnd': [], 'TCA_rnd_precision': [],
 'TCA_rnd_recall': [], 'TCA_rnd_accuracy': [], 'LT': [], 'LT_precision': [], 'LT_recall': [], 'LT_accuracy': [], 'Dycom': [], 'Dycom_precision': [],
 'Dycom_recall': [], 'Dycom_accuracy': [], 'TDS': [], 'TDS_precision': [], 'TDS_recall': [], 'TDS_accuracy': []}

error_results = {'MCW': [], 'comb': [], 'TPTL': [], 'TCA_rnd': [], 'LT': [], 'Dycom': [], 'TDS': []}
ss = KFold(n_splits=NUM_OF_FOLDS,shuffle=True,random_state=14)

fold = 0
for train_index, test_index in ss.split(X,y):
    fold += 1
    scaler = StandardScaler()
    x_train, y_train = X.iloc[train_index], y.iloc[train_index]
    x_test, y_test = X.iloc[test_index], y.iloc[test_index]
    sm = SMOTE(random_state=random_st, k_neighbors=2)
    X_samp, y_samp = sm.fit_resample(x_train, y_train)
    print("Oversampling is finished....")
    # building the model
    rfc = grid_dictionary[META_CLASSIFIER][0]
    param_grid = grid_dictionary[META_CLASSIFIER][1]

    CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
    CV_rfc.fit(X_samp, y_samp)

    preds = CV_rfc.best_estimator_.predict(x_test)
    print(CV_rfc.best_estimator_)
    acc_list_embd.append(accuracy_score(y_test, preds))
    f1_list.append(f1_score(y_test, preds, average='micro'))

    ####### RESULTS ANALYSIS #######
    with_res = results_processing(algo_list, embedding_features, y_test, preds)
    
    for algo in ['MCW', 'comb', 'TPTL', 'TCA_rnd', 'LT', 'Dycom', 'TDS']:
        mean_results_embd[algo].append(np.mean(with_res[algo]))
        mean_results_embd[algo + "_precision"].append(np.mean(with_res[algo + "_precision"]))
        mean_results_embd[algo + "_recall"].append(np.mean(with_res[algo + "_recall"]))
        mean_results_embd[algo + "_accuracy"].append(np.mean(with_res[algo + "_accuracy"]))
        error_results[algo].append(np.mean(with_res[algo + "_error"]))
    mean_results_embd['best'].append(np.mean(with_res['best']))
    # results_all = with_res.describe()
    with_res.to_csv(f"Results_meta_model/current_results/embedding_result_{fold}.csv")
    
print('random_state:', random_st)
print("MCW f1: ", np.mean(mean_results_embd['MCW']), ", MCW acc: ", np.mean(mean_results_embd['MCW_accuracy']),
      ", MCW precision: ", np.mean(mean_results_embd['MCW_precision']), ", MCW recall: ",
      np.mean(mean_results_embd['MCW_recall']))
print("comb f1: ", np.mean(mean_results_embd['comb']), ", comb acc: ", np.mean(mean_results_embd['comb_accuracy']),
      ", comb precision: ", np.mean(mean_results_embd['comb_precision']), ", comb recall: ",
      np.mean(mean_results_embd['comb_recall']))
print("best: ", np.mean(mean_results_embd['best']))
print("accuracy meta model list: " , acc_list_embd)

Oversampling is finished....
GradientBoostingClassifier(loss='deviance', max_features='sqrt',
                           min_samples_leaf=0.1, min_samples_split=0.28,
                           random_state=14, subsample=0.8, warm_start=True)
Oversampling is finished....
GradientBoostingClassifier(loss='deviance', max_features='sqrt',
                           min_samples_leaf=0.1, min_samples_split=0.28,
                           random_state=14, subsample=0.8, warm_start=True)
Oversampling is finished....
GradientBoostingClassifier(loss='deviance', max_features='sqrt',
                           min_samples_leaf=0.1, min_samples_split=0.28,
                           random_state=14, subsample=0.8, warm_start=True)
Oversampling is finished....
GradientBoostingClassifier(loss='deviance', max_features='sqrt',
                           min_samples_leaf=0.1, min_samples_split=0.28,
                           random_state=14, subsample=0.8, warm_start=True)
Oversampling is finished....

In [18]:
pd.DataFrame(mean_results_embd)

,best,MCW,MCW_precision,MCW_recall,MCW_accuracy,comb,comb_precision,comb_recall,comb_accuracy,TPTL,...,LT_recall,LT_accuracy,Dycom,Dycom_precision,Dycom_recall,Dycom_accuracy,TDS,TDS_precision,TDS_recall,TDS_accuracy
0,0.308862,0.284735,0.273869,0.366862,0.891715,0.298428,0.342619,0.368449,0.856584,0.164394,...,0.150174,0.901518,0.112992,0.270057,0.123670,0.748458,0.143098,0.543317,0.099247,0.616961
1,0.359682,0.329853,0.345099,0.381157,0.888389,0.348517,0.421455,0.383291,0.854951,0.181445,...,0.159233,0.892166,0.114811,0.279202,0.119629,0.731632,0.150781,0.546699,0.107920,0.612172
2,0.328039,0.295878,0.307718,0.345178,0.877619,0.320451,0.408509,0.352427,0.833854,0.184204,...,0.195457,0.898608,0.117727,0.275406,0.130782,0.738622,0.153226,0.528190,0.106986,0.624451
3,0.303277,0.267051,0.252118,0.344341,0.882824,0.281708,0.322132,0.345987,0.844795,0.171455,...,0.151402,0.904194,0.102899,0.259948,0.109844,0.738786,0.156970,0.542270,0.107129,0.634979
4,0.321101,0.291739,0.302515,0.340292,0.880600,0.301746,0.385347,0.340332,0.838211,0.172734,...,0.139748,0.900435,0.100799,0.250994,0.132890,0.757159,0.155315,0.575606,0.104425,0.611512


In [19]:
res_unioned = []
for i in range(1,6):
    temp = pd.read_csv(f'Results_meta_model/current_results/embedding_result_{i}.csv')
    res_unioned.append(temp)
res_pd = pd.concat(res_unioned)
res_pd.to_csv('embedding_results.csv')